<a target="_blank" href="https://colab.research.google.com/github/Jasonchen0317/CSGY-6613-Assignment/blob/main/assignment-3/assignment-3.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00


In [2]:
from pytube import YouTube
import cv2
import os

import numpy as np
import tqdm
import pandas as pd


In [7]:
def download_video(video_list):
    for i in range(len(video_list)):
        #get link
        link = video_list[i]
        yt = YouTube(link)
        yt.bypass_age_gate()
        #Download video
        filename = "%s.mp4" % str(i)
        yt.streams.filter(res="720p").first().download(filename=filename, output_path='video')

In [8]:
if not os.path.exists('video'):
   os.makedirs('video')

video_list = [
    'https://www.youtube.com/watch?v=DhmZ6W1UAv4',
    'https://www.youtube.com/watch?v=YrydHPwRelI'
]

In [9]:
download_video(video_list)

In [12]:
v_list = os.listdir('video')
for i in range(len(v_list)):
  vid = i
  capture = cv2.VideoCapture(f'video/{vid}.mp4')
  length = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
  print( length )

4941
15409


In [3]:
if not os.path.exists('dataset/test'):
   os.makedirs('dataset/test')

In [13]:
DIR_TEST = 'dataset/test'
for i in range(len(v_list)):
  vid = i
  capture = cv2.VideoCapture(f'video/{vid}.mp4')
  frameNr = 0
  success, frame = capture.read()
  while (success):
      cv2.imwrite(f'{DIR_TEST}/video_{vid}_frame_{frameNr}.jpg', frame)
      success, frame = capture.read()
      frameNr = frameNr+1

  capture.release()


In [1]:
import torch
torch.cuda.is_available()

True

In [5]:
torch.__version__
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='NVIDIA GeForce RTX 4050 Laptop GPU', major=8, minor=9, total_memory=6140MB, multi_processor_count=20)

In [3]:
!nvidia-smi

Thu Nov  9 12:25:00 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.67                 Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   52C    P0              22W /  44W |   5922MiB /  6141MiB |     98%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
if not os.path.exists('outputs'):
   os.makedirs('outputs')

In [ ]:
!python train.py

In [ ]:
if not os.path.exists('detections'):
   os.makedirs('detedtions')

In [ ]:
!python inference.py